<a href="https://colab.research.google.com/github/youneslebboukh/text-summarization/blob/main/txt_summ_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"youneslbk","key":"ff30355568a25aef64e5c064a14e36fc"}'}

In [3]:
 !mkdir -p ~/.kaggle
 !cp kaggle.json ~/.kaggle/

In [4]:
  !chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge

100% 6.21G/6.22G [01:30<00:00, 55.6MB/s]
100% 6.22G/6.22G [01:30<00:00, 73.6MB/s]


In [6]:
!pip install zip_files

In [7]:
from zipfile import ZipFile
file_name="CORD-19-research-challenge.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [8]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
metadata = pd.read_csv("/content/metadata.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
metadata.head(3)

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [12]:
import json
import glob

In [14]:
p="/content/document_parses"
all_json = glob.glob(f'{p}/**/*.json', recursive=True)
len(all_json)

241502

In [16]:
full_body_text=[]
for i in range(5) :
  with open(all_json[i]) as file :
    content = json.load(file)
    abstract = []
    body_text = []
    # Body text
    for entry in content['body_text']:
      body_text.append(entry['text'])
    body_text = '\n'.join(body_text)
    full_body_text.append(body_text)
    # Abstract
    #if 'abstract' in content:
      #for entry in content['abstract']:
        #abstract.append(entry['text'])
      #abstract = '\n'.join(abstract)
    

In [18]:
for i in full_body_text :
  print(i)
print(len(full_body_text)) 

Low level laser therapy (LLLT) is also known as cold laser therapy or photobiomodulation therapy. LLLT utilizes visible light and infrared laser beams in the range of 450–1000 nm. Single wavelength or monochromatic light is emitted from a low-intensity laser diode (<500 mW). The light source is placed in contact with the skin, allowing the photon energy to penetrate tissue, where it interacts with various intracellular biomolecules to restore normal cell function and enhance the body’s healing processes [1]. This contrasts with the thermal effects produced by the high-power lasers that are used in cosmetic and surgical procedures to destroy tissue [1], as mentioned in the PubMed Medical Subject Heading (MeSH) subheading for LLLT.
LLLT effects are not due to heat but rather to a photochemical reaction that occurs when a photoacceptor molecule within the cell absorbs a photon of light, becomes activated, and changes the cell’s membrane permeability and metabolism. Presently, cytochrome c

In [19]:
def preprocessing(text):
    # remove mail
    text = re.sub(r'[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}', 'MAIL', text)
    # remove doi
    text = re.sub(r'https\:\/\/doi\.org[^\s]+', 'DOI', text)
    # remove https
    text = re.sub(r'(\()?\s?http(s)?\:\/\/[^\)]+(\))?', '\g<1>LINK\g<3>', text)
    # remove single characters repeated at least 3 times for spacing error (e.g. s u m m a r y)
    text = re.sub(r'(\w\s+){3,}', ' ', text)
    # replace tags (e.g. [3] [4] [5]) with whitespace
    text = re.sub(r'(\[\d+\]\,?\s?){3,}(\.|\,)?', ' \g<2>', text)
    # replace tags (e.g. [3, 4, 5]) with whitespace
    text = re.sub(r'\[[\d\,\s]+\]', ' ',text)
     # replace tags (e.g. (NUM1) repeated at least 3 times with whitespace
    text = re.sub(r'(\(\d+\)\s){3,}', ' ',text)
    # replace '1.3' with '1,3' (we need it for split later)
    text = re.sub(r'(\d+)\.(\d+)', '\g<1>,\g<2>', text)
    # remove all full stops as abbreviations (e.g. i.e. cit. and so on)
    text = re.sub(r'\.(\s)?([^A-Z\s])', ' \g<1>\g<2>', text)
    # correctly spacing the tokens
    text = re.sub(r' {2,}', ' ', text)
    text = re.sub(r'\.{2,}', '.', text)
    text=re.sub(r"[<>()(,)|&©ø\[\]\'\";?~*!]", ' ', text) #remove <>()|&©ø"',;?~*!
    text=re.sub("(\\t)", ' ', text) #remove escape charecters
    text=re.sub("(\\r)", ' ', text) 
    text=re.sub("(\\n)", ' ', text)
    text= re.sub("(\s+)",' ',text) #remove multiple space
    # return lowercase text
    return text.lower()

In [20]:
from nltk.tokenize import sent_tokenize
full_body_text2=[]

In [21]:
for i in full_body_text :
  txt = preprocessing(i)
  t=sent_tokenize(txt)
  full_body_text2.append(t)

In [22]:
for s in full_body_text2 :
  print(s)

['low level laser therapy lllt is also known as cold laser therapy or photobiomodulation therapy.', 'lllt utilizes visible light and infrared laser beams in the range of 450–1000 nm.', 'single wavelength or monochromatic light is emitted from a low-intensity laser diode 500 mw .', 'the light source is placed in contact with the skin allowing the photon energy to penetrate tissue where it interacts with various intracellular biomolecules to restore normal cell function and enhance the body’s healing processes .', 'this contrasts with the thermal effects produced by the high-power lasers that are used in cosmetic and surgical procedures to destroy tissue as mentioned in the pubmed medical subject heading mesh subheading for lllt.', 'lllt effects are not due to heat but rather to a photochemical reaction that occurs when a photoacceptor molecule within the cell absorbs a photon of light becomes activated and changes the cell’s membrane permeability and metabolism.', 'presently cytochrome 

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [25]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [29]:
clean_body_text = []
for r in full_body_text2 :
  c_body_text = []
  for rr in r :
    c_body_text.append(remove_stopwords(rr.split()))
  clean_body_text.append(c_body_text)

for s in clean_body_text :
  print(s)

['low level laser therapy lllt also known cold laser therapy photobiomodulation therapy.', 'lllt utilizes visible light infrared laser beams range 450–1000 nm.', 'single wavelength monochromatic light emitted low-intensity laser diode 500 mw .', 'light source placed contact skin allowing photon energy penetrate tissue interacts various intracellular biomolecules restore normal cell function enhance body’s healing processes .', 'contrasts thermal effects produced high-power lasers used cosmetic surgical procedures destroy tissue mentioned pubmed medical subject heading mesh subheading lllt.', 'lllt effects due heat rather photochemical reaction occurs photoacceptor molecule within cell absorbs photon light becomes activated changes cell’s membrane permeability metabolism.', 'presently cytochrome c oxidase opsins associated calcium channels water molecules identified main mediators photochemical mechanisms .', 'leads increased mrna synthesis cell proliferation.', 'lllt produces reactive 

In [32]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in clean_body_text:
  for ss in s :
    sentences.append(sent_tokenize(ss))

sentences = [y for x in sentences for y in x] # flatten list

In [36]:
for i in sentences :
  print(i)

low level laser therapy lllt also known cold laser therapy photobiomodulation therapy.
lllt utilizes visible light infrared laser beams range 450–1000 nm.
single wavelength monochromatic light emitted low-intensity laser diode 500 mw .
light source placed contact skin allowing photon energy penetrate tissue interacts various intracellular biomolecules restore normal cell function enhance body’s healing processes .
contrasts thermal effects produced high-power lasers used cosmetic surgical procedures destroy tissue mentioned pubmed medical subject heading mesh subheading lllt.
lllt effects due heat rather photochemical reaction occurs photoacceptor molecule within cell absorbs photon light becomes activated changes cell’s membrane permeability metabolism.
presently cytochrome c oxidase opsins associated calcium channels water molecules identified main mediators photochemical mechanisms .
leads increased mrna synthesis cell proliferation.
lllt produces reactive oxygen species ros normal 

In [33]:
#Download GloVe Word Embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2020-12-14 17:56:55--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-14 17:56:56--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-14 17:56:56--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [34]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [35]:
len(word_embeddings)

400000

In [37]:
sentence_vectors = []
for i in sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [38]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [41]:
#Applying PageRank Algorithm
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [42]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [43]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

although pfu titer indicates functional measurement rather quantitive method higher detection limits 0 1 1 pfu obtained almost viruses 2007 isolates re-emerged since november 2005. two strains showed lower sensitivity rt-lamp method also belonged group.
3a dimer added 100 μm emodin initial processing similar dimer without added drug see processing supplement .
findings justify use il-6 receptor antagonists however therapy reduce inflammation multiple levels lllt could successful controlling unbalanced immune response figure 1 .
however sensitivity lamp uses six primers recognizing eight distinct regions target gene stable due rapid changes h5n1 influenza virus genotypes.
notably positive samples whose detection time around 20 minutes could propagated cell culture.
evolution analysis updated h5n1 viruses characterized vietnam’s strains belonging subclade 2 3 4 sequence alignment strains shows target region rt-lamp still well conserved.
second assay run modular wuhan cov rdrp gene result